In [6]:
import numpy as np
import os
import cv2
import math

def import_images(path):
    imgs = [cv2.imread(os.path.join(path, img_name), 0) for img_name in os.listdir(path)]
    resize_all(imgs)
    return np.array(imgs)

def import_target(path):
    f = open(os.path.join(path, "target.csv"))
    lines = f.read().split('\n')
    toks = [line.split(',') for line in lines]
    toks = toks[1:-1]
    return np.array([int(float(tok[2])) for tok in toks])

def import_data():
    X_all = import_images(os.path.join("images"))
    Y_all = import_target("")
    Y_all = Y_all[:len(X_all)]
    return X_all, Y_all

def resize_all(X):
    return np.array([cv2.resize(x,dsize=(200,150)) for x in X])

def drop_even(X, Y, M, N):
    indices = []
    for i in range(len(Y)):
        if (Y[i] == 1 and M > 0):
            M -= 1
            indices.append(i)
        if (Y[i] == 0 and N > 0):
            N -= 1
            indices.append(i)
    return X[indices], Y[indices]

def split_even(X, Y, r):
    train_inds = []
    test_inds = []
    N = int(len(Y)*r) # the number of objects that will go into training dataset
    N0, N1 = int(N/2), int(N/2)
    for i in range(len(Y)):
        if (Y[i] == 0 and N0 > 0):
            N0 -= 1
            train_inds.append(i)
        elif (Y[i] == 1 and N1 > 0):
            N1 -= 1
            train_inds.append(i)
        else:
            test_inds.append(i)
    return X[train_inds], Y[train_inds], X[test_inds], Y[test_inds]

In [2]:
X, Y = import_images(os.path.join("images")), import_target("")
X = np.reshape(X, (10015, 150, 200, 1))
X_d, Y_d = drop_even(X, Y, 3310, 3310)
X_train, Y_train, X_test, Y_test = split_even(X_d,Y_d,0.8)

Grey images

In [19]:
from keras import layers
from keras import models
from keras import optimizers
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 200, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 198, 32)      320       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 74, 99, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 72, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 36, 48, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 34, 46, 128)       73856     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 17, 23, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 15, 21, 128)       147584    
__________

In [20]:
model.fit(X_train, Y_train, epochs=5, batch_size=64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
5296/5296 [==============================] - 405s 77ms/step - loss: 7.9890 - acc: 0.5000
Epoch 2/5
3072/5296 [================>.............] - ETA: 3:46 - loss: 8.2532 - acc: 0.4880

KeyboardInterrupt: 